<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/4_Divide_y_venceras_con_estructuras_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta sesión, veremos dos estructuras de datos que están muy ligadas a las ideas de los algoritmos de divide y vencerás, que suelen ser muy útiles para realizar operaciones que suelen requerirse bastante.

**Trie.** Consideremos el siguiente problema. Dadas dos palabras, decimos que su "similitud" es el mayor número de letras iniciales en las que coinciden, es decir, es el tamaño del prefijo común más grande entre ambas palabras. Dado un conjunto de palabras $C$, ,y una nueva palabra, $s$, queremos saber la mayor similitud posible con alguna letra del conjunto. 

¿Cómo proceder? Una forma de hacerlo es haciendo una búsqueda exhaustiva, donde para cada palabra de $C$ la comparamos con la palabra $s$, y vamos verificando letra por letra los prefijos hasta que dejen de coincidir. Podemos acotar la complejidad en tiempo por $O(|C|\cdot |s|)$. ¿Qué pasa si queremos repetir el proceso para una cantidad muy grande de palabras s?

Otra forma de atacar el problema sería ordenar las palabras en orden alfabético, incluyendo a $s$, y posteriormente comparar a $s$ con las palabras vecinas en el ordenamiento (queda como ejercicio probar que en efecto uno de sus vecinos es la palabra que cumple tener mayor similitud con $s$). ¿Qué complejidad tiene esto?

La forma que veremos introduce una estructura, que resulta ser muy poderosa para este tipo de problemas, es un árbol especial, que tiene por nombre *trie*. Un trie es un árbol donde cada nodo puede tener $26$ hijos (uno por cada letra del abecedario). De este modo, si una palabra tiene las letras "xy" de forma consecutiva, el respectivo nodo tendrá un hijo no nulo en la entrada correspondiente a "y". Veamos un ejemplo de esta estructura.

Consideremos las palabras `[abcdef, abecede, hola, hoja, puedes, luz]`. El trie respectivo a este conjunto de palabras es el siguiente:


       (Nodo raíz)
      /   |   |  \
     a    h   p   l
     |    |   \    \
     b    o    u    u
    / \   | \   \    \
   c   e  l  j   e    z
   |   |  |  |   |
   d   c  a  a   d
   |   |         |
   e   e         e
   |   |         |
   f   d         s
       |
       e


Esta estructura nos permite identificar de manera rápida y sencilla el mayor prefijo que coincide con cierta palabra. Veamos cómo implementar esta estructura y cómo usarla para nuestro problema.

In [ ]:
class TrieNode:
  def __init__(self):
    self.children = [None]*26

def insert(root, word): # Inserta una nueva palabra en el Trie con raíz root
  l = len(word)
  curr = root
  for i in range(0, l):
    aux = ord(word[i]) - ord('a') # Recorremos los índices para empezar en 0
    if not curr.children[aux]:
      curr.children[aux] = TrieNode()
    curr = curr.children[aux]
    

def prefix(root, word): # Nos permite encontrar el mayor prefijo de s en C
  curr = root
  cnt = 0
  l = len(word)
  for i in range(0,l):
    aux = ord(word[i]) - ord('a')
    if not curr.children[aux]:
      return cnt
    else:
      curr = curr.children[aux]
      cnt += 1
  return cnt


C = ["abcdef", "abecede", "hola", "hoja", "puedes", "luz"]
Troot = TrieNode()
for c in C:
  insert(Troot, c)

S = ["abaco", "holandes", "paz"]
for s in S:
  print(prefix(Troot, s))


2
4
1


**Segment tree.** Dada una lista $L$ de números reales ($|L| = n$), queremos saber la suma de los elementos en un intervalo dado, es decir $L[i] + L[i+1] + \dots + L[j]$ para determinados $i \leq j$. Además de esto, queremos poder actualizar nuestra lista $L$. Es decir, tenemos dos tipos de operaciones:

*   Preguntar por la suma de los valores con índices en el intervalo $[i,j]$.
*   Actualizar el valor de cierto elemento $L[i]$.

Una forma sencilla de poder recuperar el valor de la suma en cierto intervalo es guardando una lista de sumas acumuladas, sin embargo, esto hace que las operaciones de actualización de elementos sean costosas.

*Descomposición en raíz cuadrada*. ¿Qué pasa si dividimos la lista en sublistas de longitud $m$? (Veremos después qué $m$ nos conviene). Tendremos un total de $\frac{n}{k}$ 'bloques', para cada bloque guardaremos el valor de la suma de sus elementos. ¿Qué tan costosas son las operaciones de actualización y preguntar por sumas? Notemos que para actualizar, únicamente debemos actualizar el valor del elemento, así como la suma correspondiente en el bloque de tamaño $k$, por lo que nos toma $O(1)$ cada actualización. Para calcular la suma de un intervalo dado, si tenemos que cierto bloque está completamente contenido en el intervalo, procedemos a considerar la suma previamente guardada (ya no accesamos a los elementos correspondientes), por lo que a lo más accederemos dentro de dos bloques, el extremo izquierdo y el extremo derecho. Como tenemos a lo más $\frac{n}{k}$ bloques, y posteriormente tenemos que calcular las sumas para los bloques 'parcialmente' contenidos, esto nos toma $O(\frac{n}{k} + k)$. 

Queremos entonces un valor de $k$ que minimice $\frac{n}{k} + k$, utilizando $MA-MG$, obtenemos que $\frac{n}{k} + k \geq 2 \sqrt{n}$, con igualdad si $k = \sqrt{n}$. Es por esto que consideraremos bloques de tamaño $\sqrt{n}$. 

Entonces, hemos logrado una forma de resolver el problema de modo que actualizar nos tome $O(1)$, mientras que preguntar por suma en determinado intervalo $O(\sqrt{n})$.



*Segment tree.* Una forma de optimizar el tiempo de preguntar por un intervalo (que hará que actualizar sea más costoso, pero no mucho más), es utilizando un árbol binario, donde el nodo padre tiene el valor de la suma de sus nodos hijos, así como dos enteros asociados, el extremo izquierdo y el extremo derecho del intervalo que abarca. Y donde los nodos que son hojas corresponden a la lista original $L$.

De este modo, logramos que preguntar por la suma en un intervalo $[l,r]$ sea $O(logn)$, mientras que actualizar también nos tomará $O(logn)$, ya que esta es la altura de nuestro árbol.

Veamos una implementación de esta estructura y un ejemplo de su uso.

In [ ]:
import random

L = [random.randrange(0, 10, 1) for i in range(36)]

class SegNode():
  def __init__(self):
    self.value = 0 # 0 será nuestro valor por default
    self.left = None
    self.right = None
    self.start = 0
    self.end = 0

def build(l, r): # Construye el segment tree con extremos l y r
  if(l > r):
    return None
  new = SegNode()
  new.start = l
  new.end = r
  if(l == r):
    new.value = L[l]
  else:
    m = (l+r)//2
    new.left = build(l,m)
    new.right = build(m+1, r)
    if(not new.left == None):
      new.value += new.left.value
    if(not new.right == None):
      new.value += new.right.value
  return new

def query(root, l, r): # Función que nos permite preguntar por la suma en el intervalo l,r
  if(root is None):
    return 0
  if(l <= root.start and root.end <= r):
    return root.value
  if(root.end < l):
    return 0
  if(r < root.start):
    return 0
  return query(root.left, l, r) + query(root.right, l, r)

def update(root, i, val): # Función que nos permite poner el valor val en la posición i
  L[i] = val
  if(root is None):
    return 0
  if(root.end < i or i < root.start): # Si i no está en el intervalo dado
    return root.value
  if(root.end == root.start and root.start == i):
    root.value = val
  else:
    root.value = update(root.left, i, val) + update(root.right, i, val)
  return root.value

print(L)
Stree = build(0, len(L)-1)
print(query(Stree, 0, 35))
print(query(Stree, 2, 4))
update(Stree, 2, 5)
print(L)
print(query(Stree, 2, 4))
print(query(Stree, 10, 20))
update(Stree, 15, 4)
print(L)
print(query(Stree, 10, 20))
print(query(Stree, 0, 35))

[1, 0, 6, 5, 4, 7, 6, 2, 1, 2, 7, 2, 1, 3, 5, 7, 5, 2, 0, 9, 4, 4, 6, 3, 8, 5, 7, 4, 3, 0, 1, 4, 1, 2, 6, 5]
138
15
[1, 0, 5, 5, 4, 7, 6, 2, 1, 2, 7, 2, 1, 3, 5, 7, 5, 2, 0, 9, 4, 4, 6, 3, 8, 5, 7, 4, 3, 0, 1, 4, 1, 2, 6, 5]
14
45
[1, 0, 5, 5, 4, 7, 6, 2, 1, 2, 7, 2, 1, 3, 5, 4, 5, 2, 0, 9, 4, 4, 6, 3, 8, 5, 7, 4, 3, 0, 1, 4, 1, 2, 6, 5]
42
134


**Ejercicios.** 

1.   Utilizando la estructura de Trie, dado un conjunto de palabras $C$, determina si una palabra $s$ está o no en el conjunto $C$. Prueba tu código con $C = \{ola, hola, noche, diez, no, ciencia, camaron\}$ y las palabras $cama, hola, nop, cien, diez$. (Hint: Agrega a la estructura un valor que nos permita identificar si cierto nodo corresponde al final de una palabra o no).
2.   Utilizando descomposición en raíz cuadrada. Implementa un algoritmo que permita actualizar elementos y consultar sumas en determinados intervalos para una lista de $36$ elementos. Compara los resultados de las consultas con el resultado que se obtiene al usar un segment tree.



*Ejercicio 1.* Escribe a continuación el código solicitado

In [ ]:
# Aquí va el código del ejercicio 1

*Ejercicio 2.* Escribe a continuación el código solicitado.

In [ ]:
# Aquí va el código del ejercicio 1